## Question 1.  Social Media and Study Results

We  want  to  investigate  whether  there  is  a  link  between  the  use  of  social  media  and  the  performance  of
students.  We  ask  1064  students  how  many  hours  they  spend  on  average  per  day  using  social  media.    By
social media we  mean Facebook, YouTube, blogs, Twitter,  MySpace and LinkedIn.  We also check the final
score  of  these  students  at  the  end  of  the  year.  The  result  of  this  survey  can  be  found  in  the  file
"socialMediaVsPoints.csv".

a. Load the file.  Ensure that the number of hours is read as numbers.  Remove the rows with an
unknown value.  Now remove the rows with outliers. How many rows are left?
b. Make a histogram of the hours and the points separately.
c. What is the average number of hours a student spends on social media?
d. What is the standard deviation of the number of hours?  What does this mean?
e. What is the average score of the students?
f. What is the standard deviation of the score?
g. Make a scatterplot of the 2 variables.  What correlation do you expect?
h. Calculate the correlation between the 2 variables.  Do this also via the Z-scores (Pearson).  What did
you decide?
i. Which rank correlation coefficient do you find according to Kendall?  What does this value mean?
j. Does it make sense to determine a regression line here?  Why yes/no?  Determine the regression
line if it makes sense.

In [ ]:
import pandas as pd
sm = pd.read_csv('socialMediaVsPoints.csv',delimiter=',',decimal='.',na_values="unknown")
sm

In [ ]:
sm=sm.dropna()
sm['hours']=pd.to_numeric(sm['hours'])
sm

In [ ]:
import numpy as np
def remove_outliers(data):
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    I = Q3 - Q1
    low = Q1 - 1.5 * I
    high = Q3 + 1.5 * I
    return data[data.between(low,high)]
sm['hours'],sm['points']=remove_outliers(sm['hours']),remove_outliers(sm['points'])
sm

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
sm['hours'].plot.hist()
plt.show()

In [ ]:
plt.figure()
sm['points'].plot.hist()
plt.show()

In [ ]:
sm['hours'].mean()

In [ ]:
sm['hours'].std()

In [ ]:
sm['points'].mean()

In [ ]:
sm['points'].std()

In [ ]:
plt.figure()
plt.scatter(sm['hours'],sm['points'])
plt.show()

In [ ]:
sm.corr(method='pearson')

In [ ]:
sm['hours'].corr(sm['points'])

In [ ]:
sm.corr(method='kendall')

no correlation
## QUESTION 2
Batteries for smartphones are manufactured in a factory.  The demand for these batteries varies considerably.
One suspects that if demand (and therefore production) increases, the percentage of exploding batteries also
increases (and thus: the quality drops due to the working pressure).  One wants to verify this.

A  measurement  is  made.    Every  day  the  number  of  batteries  produced  is  tracked.    If  a  defective  battery  is
returned  (or  one  explodes),  it  is  verified  on  which  day  it  was  produced.    The  data  can  be  found  in  the  file
"batteries.csv". The first column shows how many batteries were produced and the second column shows how
many batteries were defective.

a. Load the file.  There are lines with more than 2 values.  See what Python does with them when you
load the values.  Remove this line from the data.  There are outliers in the column Defect and
remove these.  How many rows do you have left now?
b. Make a scatterplot of the two variables.  Is there a linear correlation?
c. What correlation do you find with Kendall's method?
d. We are actually not interested in the absolute number of defective batteries, but we are interested in
the percentage.  So divide the number of defective batteries by the number of batteries produced.
What is the average percentage of defective batteries?
e. Make a scatterplot with the percentage of defective batteries.  Can you see the difference with the
previous graph?
f. Now calculate the correlation between the number of batteries produced and the percentage of
defective batteries.  What do you decide?
g. What is the value of R²?  What does it mean?
h. Draw the regression line on the scatterplot.  What are the values for the slope and intercept?
i. What is the standard estimated error?  What does this mean?
j. If production would be increased to 8000 batteries/day, how many defective batteries would you
expect then?
k. How many batteries can the factory produce per day so that there is no more than 1 percent defect?

In [ ]:
import numpy as np
bt = pd.read_csv('batteries.csv',delimiter=',',na_values=np.nan)
bt

In [ ]:
bt['#Defects']=remove_outliers(bt['#Defects'])
bt['#Defects']=pd.to_numeric(bt['#Defects'])
bt=bt.dropna()
bt

In [ ]:
plt.figure()
plt.scatter(bt['#Produced'],bt['#Defects'])
plt.show()

In [ ]:
bt.corr(method='kendall')

In [ ]:
bt['#Produced'].sum()/bt['#Defects'].sum()

In [ ]:
btp=bt
btp['percentage']=(btp['#Defects']/btp['#Produced']) *100
plt.figure()
plt.scatter(btp['#Produced'],btp['percentage'])
plt.show()
btp

In [ ]:
btp['#Produced'].corr(btp['percentage'])

In [ ]:
import math
def general_regression(x, y, degree=1, exp=False):
    data = pd.DataFrame({'x':x,'y':y})
    data.reset_index(drop=True, inplace=True)
    func=lambda x:x # def func(x): return[x]
    inv_func=lambda x:x
    if (exp):
        func=np.exp
        inv_func=np.log
    sy = data.y.std()
    model = np.polyfit(x, inv_func(y), degree)
    line = np.poly1d(model)
    predict = lambda x:func(line(x))
    data['y_pred'] = pd.Series(predict(x))
    se = math.sqrt(((data.y_pred-data.y)**2).mean())
    R2 = 1-(se**2)/(sy**2)
    result = [se, R2, predict]
    index = ['se', 'R2', 'predict']
    for i in range(1,len(model)+1):
        result = np.append(result, model[-i])
        index += chr(i+96) # to obtain the characters a,b,...
    result = pd.Series(result)
    result.index = index
    return result
general_regression(btp['#Produced'],btp['percentage'],1)

In [ ]:
def draw_linear_regression_scatter_plot(x: pd.Series, y: pd.Series, x_label: str, y_label: str, title: str) -> None:
    xx, yy, se = linear_regression(x, y)
    plt.scatter(x, y, alpha=0.2)
    plt.plot(xx, yy, color='red')
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)
    plt.fill_between(xx, yy - 2 * se, yy + 2 * se, color='#FFFF0020')
    plt.show()
draw_linear_regression_scatter_plot(btp['#Produced'],btp['percentage'],"produced","percentage","QH")